In [6]:
! pip install gensim
! pip install jieba

  Stored in directory: C:\Users\Hannah\AppData\Local\pip\Cache\wheels\af\e4\8e\5fdd61a6b45032936b8f9ae2044ab33e61577950ce8e0dec29
Successfully built jieba


In [14]:
import re
import os
import numpy as np
import pandas as pd
import jieba
import jieba.analyse  
import matplotlib.pyplot as plt
from tqdm import tqdm

import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors

In [2]:
all_text = pd.read_csv('all_text.csv')
all_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94043 entries, 0 to 94042
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   length_text          94043 non-null  int64 
 1   length_retweet_text  94043 non-null  int64 
 2   text_pro             94043 non-null  object
 3   retweet_text_pro     41922 non-null  object
dtypes: int64(2), object(2)
memory usage: 3.6+ MB


In [3]:
all_text.head()

,length_text,length_retweet_text,text_pro,retweet_text_pro
0,79,0,这是不是无耻到一定境界了？抗击暴雨的普通百姓，居然是北京精神的作用？人们的团结热心，居然是某...,NaN
1,6,142,对比北京暴雨,【俄罗斯洪灾 3名官员因失职被捕】俄罗斯南部于今年7月7日，遭暴雨袭击并引发洪灾，亡人数达1...
2,24,0,"北京市民坚持住!众志成城,定可度过暴雨阶段!!!",NaN
3,48,0,北京暴雨死了这么多人，不但新闻上不了头条，连主持人都穿着这么鲜艳的衬衣。真是一个毫无人性的国家！,NaN
4,102,124,大眼，好样的！从什邡到北京暴雨，那些公知们忙着晒裸照，约口水战粉饰太平时，很高兴看到还有你这...,山后再铁《图腾》——“这几乎是个图腾，只有上善若水，没有从善如流，不是城市不足，而是成事不足...


筛选原创文本，即转发文本长度length_retweet_text为0的文本

In [4]:
# 先处理原创微博文本
print('all_text所有文本的数目为：', len(all_text))
df_text = all_text[all_text['length_retweet_text']==0][['length_text','text_pro']].reset_index(drop=True)
df_text.drop_duplicates(subset='text_pro',inplace=True)
df_text = df_text.reset_index(drop=True)
print('原创微博文本的数目为：', len(df_text))

text = df_text['text_pro']
df_text

# pattern_text = r'// 分享网易新闻:*|// 分享网易新闻:「*|//分享网易*'
# text = [re.sub(pattern_text,' ',df_text['text_pro'][i]) for i in range(len(df_text['text_pro']))]

all_text所有文本的数目为： 94043
原创微博文本的数目为： 47390


,length_text,text_pro
0,79,这是不是无耻到一定境界了？抗击暴雨的普通百姓，居然是北京精神的作用？人们的团结热心，居然是某...
1,24,"北京市民坚持住!众志成城,定可度过暴雨阶段!!!"
2,48,北京暴雨死了这么多人，不但新闻上不了头条，连主持人都穿着这么鲜艳的衬衣。真是一个毫无人性的国家！
3,120,@中国泛户外第一人李辉 @梁强户外休闲观察 户外休闲产业发展和户外灾害生存教育应该同步进行、...
4,139,截至2012年7月22日17时，北京暴雨后共发现37人死亡。1949年以来，全国的人民和财力...
...,...,...
47385,22,@下一秒_独自等待 亲，北京的雨大不？
47386,66,北京气象部门已成惊弓之鸟 每天工作第一件事就是预报有雨 管它下不下 先让大伙不出门就行 我在...
47387,74,哥回北京，果然天气放晴，哎这孩子就是不给雨… 从机场回来一路绿灯，连闯红灯的机会都不给，太没...
47388,84,雨一直下，气氛不算融洽...坝上疯行下回吧，据说城里都晴了，可是北面还是雨ing。找地吃鱼，...


评论数据清洗

In [5]:
def data_clear(all_comments,pattern_special = r'//@.*:', pattern_text = r'分享网易新闻'):
    
    # 1. 删除由特殊格式限定的广告、用户名、链接等
    x1 = all_comments
    x2 = re.sub(pattern_special," ",x1)
    
    # 2. 仅保留中文
    pattern = re.compile(r'[\u4e00-\u9fa5]+')
    filedata = re.findall(pattern, x2)
    x3 = ''.join(filedata)
    
    # 3. 去掉微博评论特有的无效信息
    new_xx = re.sub(pattern_text,'',x3)
    return new_xx

In [6]:
pattern_special = r'//@.*:|//@.*：|(来自.*)|@.*\s|http://.*'
pattern_text = r'分享网易新闻|分享|转载|微博调查|我发起了一个投票'
new_text = [ data_clear(text[i],pattern_special,pattern_text) for i in range(len(text))]

df_new_text = pd.DataFrame(data = [len(new_text[j]) for j in range(len(new_text))],columns=['length_text'])
df_new_text['new_text'] = new_text
# df_new_text.describe()

删除字符数小于15的短文本

In [7]:
print("删除短文本前评论数目：",len(df_new_text['length_text']))
new_text = df_new_text[df_new_text['length_text']>=15]['new_text'].reset_index(drop=True)
print('删除短文本后评论数目：',len(new_text))

删除短文本前评论数目： 47390
删除短文本后评论数目： 41642


In [8]:
# 筛选原创文本
# 评论数据清洗
# 删除短文本
new_text.to_csv('original_text.csv',index=False)

# 主题抽取

分词、去除停用词

In [19]:
new_text = pd.read_csv('original_text.csv')
new_text = new_text['new_text']

In [16]:
def open_stop(file_stop):
    stopwords = [line.strip() for line in open(file_stop, 'r', encoding='GBK').readlines()]
    return stopwords

    #利用jieba进行分词
def seg_sentence(sentence):
    sentence_seged = jieba.cut(sentence.strip())
    stopwords = open_stop("chineseStopWords.txt")  
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr.strip()

In [21]:
# 分词示例
sentence = new_text[0]
print(sentence)
print(seg_sentence(sentence))

这是不是无耻到一定境界了抗击暴雨的普通百姓居然是北京精神的作用人们的团结热心居然是某几位领导干部官员定下的北京精神的号召结果新华网当真无耻


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Hannah\AppData\Local\Temp\jieba.cache
Loading model cost 1.788 seconds.
Prefix dict has been built successfully.


无耻 境界 抗击 暴雨 普通百姓 居然 北京 精神 作用 团结 热心 居然 几位 领导 干部 官员 定下 北京 精神 号召 新华网 当真 无耻


In [22]:
# 批量分词 
words_in_sentence = []
for i in tqdm(range(len(new_text))):
    line_seg = seg_sentence(new_text[i])  # 这里的返回值是字符串
    words_in_sentence.append(line_seg)
words_in_sentence = [x for x in words_in_sentence if x != '']

#利用空格切割获取所有分词
alltokens = []
chinesewords_sentence = []
for i in tqdm(range(len(words_in_sentence))):
    word = re.split(r'\s',words_in_sentence[i])
    alltokens.append(word)

#对每一个句子产生的分词所储存的列表，删除空值
for element in alltokens:
    element = [x for x in element if x != '']
    chinesewords_sentence.append(element)

100%|████████████████████████████████████████████████████████████████████████| 41642/41642 [00:00<00:00, 174914.41it/s]


In [24]:
print(new_text[0],end='\n\n')
print(words_in_sentence[0],end='\n\n')
print(chinesewords_sentence[0],end='\n\n')

这是不是无耻到一定境界了抗击暴雨的普通百姓居然是北京精神的作用人们的团结热心居然是某几位领导干部官员定下的北京精神的号召结果新华网当真无耻

无耻 境界 抗击 暴雨 普通百姓 居然 北京 精神 作用 团结 热心 居然 几位 领导 干部 官员 定下 北京 精神 号召 新华网 当真 无耻

['无耻', '境界', '抗击', '暴雨', '普通百姓', '居然', '北京', '精神', '作用', '团结', '热心', '居然', '几位', '领导', '干部', '官员', '定下', '北京', '精神', '号召', '新华网', '当真', '无耻']



In [16]:
df_new_text = pd.DataFrame(new_text,columns=['new_text'])
df_new_text.head()
df_new_text.to_csv('df_new_text.csv')

In [25]:
all_comments = ""
for i in range(len(chinesewords_sentence)):
    all_comments +=  ''.join(chinesewords_sentence[i]) + ' '
print("Length of comments: ",len(all_comments))

Length of comments:  1668872


In [26]:
keywords=jieba.analyse.extract_tags(all_comments, topK=100, withWeight=True, allowPOS=('n','ns','nr'))
word_tf_idf = {x[0]:x[1] for x in keywords}

In [27]:
df_tf_idf = pd.DataFrame({'Keyword':list(word_tf_idf.keys()), 'TF_IDF':list(word_tf_idf.values())})
df_tf_idf.to_csv('df_tf_idf.csv')

In [28]:
df_tf_idf.head(5)

,Keyword,TF_IDF
0,北京,0.670162
1,暴雨,0.504059
2,大雨,0.338362
3,积水,0.128988
4,遇难者,0.071203


转换成词向量

In [29]:
file_path = 'sgns.baidubaike.bigram-char'
# 再加载第三方预训练模型：
model = gensim.models.KeyedVectors.load_word2vec_format(file_path, binary=False)

FileNotFoundError: [Errno 2] No such file or directory: 'sgns.baidubaike.bigram-char'

In [20]:
list_tfidf_word = list(word_tf_idf.keys())
vec_list = []
word_list = []
num = 0
for word in tqdm(list_tfidf_word):
    if word in model.index_to_key:
        vec = model[word]
        vec_list.append(vec)
        word_list.append(word)
    else:
        print(word)

100%|██████████| 100/100 [00:00<00:00, 1066.33it/s]

雨灾


In [21]:
from sklearn.cluster import KMeans

In [26]:
class_number = 7
clf = KMeans(n_clusters=class_number)
clf.fit(np.array(vec_list))  # 分组
labels = clf.labels_ 
use_labels = list(labels)

In [27]:
all_list=[]
for i in range(class_number):
    class_list=[]
    for j in range(len(labels)):
        if clf.labels_[j]==i:
            class_list.append(word_list[j])
    all_list.append(class_list)

for i in range(class_number):
    print('第',i+1,'类：')
    str_=''
    for word in all_list[i]:
        str_+=word
        str_+=' '
    print(str_)
    print()

第 1 类：
积水 雨量 下水道 雨水 防雨 

第 2 类：
城市 蜡烛 车门 头枕 路段 插头 玻璃 座位 雨伞 车主 车辆 动车 私家车 公司 汽车 信号 机场 童鞋 地铁 雨具 

第 3 类：
遇难者 悼念 大家 朋友 回家 蓝色 地区 逝者 新闻 老板 小时 中雨 时候 遭遇 政府 鼻屎 人民 视频 地方 奥特曼 时分 博文 人们 本市 亲们 市民 常识 群众 微风 领导 爱心 精神 问题 员工 浮云 丁先生 晚安 网友 感觉 小心 

第 4 类：
短信 电话 

第 5 类：
暴雨 大雨 降雨 强降雨 台风 天气 被淹 灾难 山洪 灾害 天灾 泥石流 洪水 暴风雨 

第 6 类：
气象台 天气预报 气象局 

第 7 类：
北京 北京市 房山 广渠门 天津 积水潭 平谷 通州 密云 中国 深圳 故宫 房山区 北京城 团城 

